In [6]:
import pandas as pd 
import numpy as np

books = pd.read_csv("data/books.csv")
tags = pd.read_csv("data/tags.csv")
book_tags = pd.read_csv("data/book_tags.csv")

## Réduire le nombre de tags par livre
La liste des tags est extrèmement longue et difficile à nettoyer (il y a de l'Arabe, du Russe, des mots avec des "-" pour les sous-catégories...). Et le tableau d'association book_tags comporte 100 000 lignes. Notre but ici est de diminuer ce nombre, en prenant seulement les tags les plus nombreux et les plus significatifs.

In [38]:
book_tags = pd.read_csv("data/book_tags.csv")

# On garde seulement les tags que plus de 110 personnes ont mis à un livre
book_tags_trimmed = book_tags[book_tags["count"]>=110]

# On crée la liste des tags à supprimer car inutile pour Content-Based Filtering, mais on les gardera pour le Popularity Based
tag_names_supp = [
    "to-read",
    "currently-reading",
    "favorites",
    "favorite",
    "books-i-own",
    "owned",
    "owned-books",
    "to-buy",
    "favourites",
    "my-books",
    "i-own",
    "re-read",
    "all-time-favorites",
    "favorite-books",
    "read-in-2008",
    "read-in-2009",
    "read-in-2010",
    "read-in-2011",
    "read-in-2012",
    "read-in-2013",
    "read-in-2014",
    "read-in-2015",
    "read-in-2016",
    "read-in-2017",
    "read-2009",
    "read-2010",
    "read-2011",
    "read-2012",
    "read-2013",
    "read-2014",
    "read-2015",
    "read-2016",
    "read-2017",
    "read-2009",
    "read-2010",
    "read-2011",
    "read-2012",
    "read-2013",
    "read-2014",
    "read-2015",
    "read-2016",
    "read-2017",
    "books-read-in-2012",
    "books-read-in-2014",
    "books-read-in-2015",
    "books-read-in-2016",
    "2012-reads",
    "2013-reads",
    "2014-reads",
    "2015-reads",
    "2016-reads",
    "2017-reads",
    ]

def get_tag_id(x):
    # Fonction qui trouve le tag_id en fonction du tag_name

    tag_id = tags.query('tag_name == "'+x+'"')['tag_id'].item()
    return tag_id

def get_tag_name(x):
    # Fonction qui trouve le tag_name en fonction du tag_id

    tag_name = tags.query('tag_id == '+str(x))['tag_name'].item()
    return tag_name

In [39]:
# On récupère les ids des tags à supprimer
tag_ids_supp=[]
 
for tag in tag_names_supp:
    tag_ids_supp.append(get_tag_id(tag))

# Boucle qui retire des book_tags les tags_ids
for tag_id in tag_ids_supp:
    book_tags_trimmed = book_tags_trimmed[book_tags_trimmed["tag_id"]!=int(tag_id)]

print("Il reste",book_tags_trimmed["tag_id"].value_counts().size,"tags et", book_tags_trimmed["goodreads_book_id"].value_counts().size, "livres")

Il reste 1864 tags et 9496 livres


In [11]:
# Les genres qu'il reste
# vc_tags_ids = pd.DataFrame(book_tags_trimmed["tag_id"].value_counts())
# vc_tags_ids = vc_tags_ids.reset_index(inplace=False)
# vc_tags_ids = vc_tags_ids.rename(columns={"tag_id":"count","index":"tag_id"})

# vc_tags_ids["tag_name"]=vc_tags_ids["tag_id"].apply(get_tag_name)
# vc_tags_ids.to_csv("data/value_counts_tags.csv")

# On garde un csv avec les tags restants, leur id, leur nom et le nombre de fois qu'ils ont été donnés

## Création du dataframe qui servira de base pour le CountVectorizer
Pour calculer la matrice de similarité entre chaque livre, il est nécessaire de créer un tableau ayant autant de lignes que de livre et autant de colonnes que de tags, et qui sera rempli de 0 ou de 1 si un certain livre est marqué d'un certain tag. Pour créer ce tableau, il nous faut utiliser la feature CountVectorizer sur un dataframe dans lequel nous aurons rentré l'id de livre ainsi qu'une chaine de carctère comprenant les ID de tous ses tags, séparés par un espace.

In [40]:
# Création d'un dataframe dans lequel une ligne correspond à un livre grâce à .unique
df = pd.DataFrame(list(book_tags_trimmed["goodreads_book_id"].unique()),columns=["book"])
df["bag_of_tags"]=" "

def create_bag(book_id):
    # Fonction qu, pour un certain livre, crée une string qui contient les ids de tous ses tags

    bag = map(str,book_tags_trimmed["tag_id"][book_tags_trimmed["goodreads_book_id"]==book_id].to_list())
    bag = " ".join(bag)

    return bag

df["bag_of_tags"] = df["book"].apply(create_bag)

In [41]:
df.head(5)

,book,bag_of_tags
0,1,11305 33114 11743 14017 32989 27199 18886 6953...
1,2,11305 6857 6888 9221 11743 33114 3389 22034 17...
2,3,11305 33114 11743 14017 32989 27199 18886 6953...
3,5,11305 33114 11743 14017 32989 27199 18886 6953...
4,6,11305 33114 11743 14017 32989 27199 6953 1691 ...


## CountVectorizer et Similarité Cosinus
Avec le dataframe créé, on peut utiliser CountVectorizer pour créer la matrice de type OneHotEncoder qui donne les tags (en colonnes) pour chaque livre (en lignes). On appelle ensuite la fonction cosine_similarity pour calculer la similarité entre les vecteurs ( = livres) en fonction de leur coordonnées ( = tag ou non, 1 ou 0)

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# A partir du bag of tags on peut utiliser countvectorizer pour créer la matrice qui nous servira a calculcer la similarité entre chaque livre
count = CountVectorizer()
count_matrice = count.fit_transform(df['bag_of_tags'])

print(count_matrice.shape)
# On vérifie que l'on a bien 9496 lignes (nb de livres) et 1864 colonnes (nb de tags possibles)

# Création de la matrice de similarité cosinus (score entre 0 et 1) entre chaque livre
cos_sim = cosine_similarity(count_matrice, count_matrice)

(9496, 1864)


## Fonction recommandation
Cette fonction recommandation Content Based prend en argument le titre d'un livre et ressort la liste de 10 livres qui lui sont le plus similaire.

In [34]:
books_title = books[["goodreads_book_id","original_title"]]
books_title.head(5)

,goodreads_book_id,original_title
0,2767052,The Hunger Games
1,3,Harry Potter and the Philosopher's Stone
2,41865,Twilight
3,2657,To Kill a Mockingbird
4,4671,The Great Gatsby


In [48]:
def get_book_id(title): 
    # Récupère l'ID du livre en fonction de son titre
    return books_title["goodreads_book_id"][books_title["original_title"]==title].item()

def get_book_title(goodreads_id):
    # Récupère le titre du livre en fonction de son id
    return books_title["original_title"][books_title["goodreads_book_id"]==goodreads_id].item()

def recommendations(title, cos_sim = cos_sim):
    """
    Fonction recommandation :
    Prend en entrée le title d'un livre et la matrice de similarité.
    Trouve l'id du livre
    Cherche dans la matrice les autres livres similaires
    """
    
    # Initialise la liste 
    recommended_books_ids = []

    # ID du livre
    book_id = get_book_id(title)
    
    # Index du livre dans la matrice
    idx = df[df["book"]==book_id].index[0]

    # Création de la série qui comprend la similiraté de notre livre et tous les autres (correspond à sa colonne de la matrice cos_sim)
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # On récupère les 10 premiers indices correspondants (plus grandes similarités)
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_books_ids.append(list(df.index)[i])
    
    recommended_books = list(map(lambda x: df["book"][x],recommended_books_ids))
    recommended_books = list(map(get_book_title, recommended_books))

    return recommended_books

In [49]:
recommendations("The Da Vinci Code")

['Angels & Demons ',
 'The Lost Symbol',
 'Män som hatar kvinnor',
 'Inferno',
 'Luftslottet som sprängdes',
 'Flickan som lekte med elden',
 'The Lovely Bones',
 'La sombra del viento',
 'Ten Little Niggers',
 'The Thirteenth Tale']